In [ ]:

import pandas as pd
import numpy as np

# Training and evaluation
import lightgbm as lgb
import xgboost as xgb
from scipy.sparse import load_npz
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

In [ ]:
# # Load transformed data from npz
X_train_transformed = load_npz(f'{base_path}\\preprocessed\\X_train_transformed.npz')
X_val_transformed = load_npz(f'{base_path}\\preprocessed\\X_val_transformed.npz')
X_test_uwarm_iwarm_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_uwarm_iwarm_transformed.npz')
X_test_uwarm_icold_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_uwarm_icold_transformed.npz')
X_test_ucold_iwarm_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_ucold_iwarm_transformed.npz')
X_test_ucold_icold_transformed = load_npz(f'{base_path}\\preprocessed\\X_test_ucold_icold_transformed.npz')

# # Load target variables
y_train = pd.read_csv(f'{base_path}\\preprocessed\\y_train.csv')
y_val = pd.read_csv(f'{base_path}\\preprocessed\\y_val.csv')
y_test_uwarm_iwarm = pd.read_csv(f'{base_path}\\preprocessed\\y_test_uwarm_iwarm.csv')
y_test_uwarm_icold = pd.read_csv(f'{base_path}\\preprocessed\\y_test_uwarm_icold.csv')
y_test_ucold_iwarm = pd.read_csv(f'{base_path}\\preprocessed\\y_test_ucold_iwarm.csv')
y_test_ucold_icold = pd.read_csv(f'{base_path}\\preprocessed\\y_test_ucold_icold.csv')

In [ ]:
# Train a LightGBM model
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train_transformed, y_train, feature_name=list(feature_names))

y_pred_uwarm_iwarm = lgb_model.predict(X_test_uwarm_iwarm_transformed)
y_pred_uwarm_icold = lgb_model.predict(X_test_uwarm_icold_transformed)
y_pred_ucold_iwarm = lgb_model.predict(X_test_ucold_iwarm_transformed)
y_pred_ucold_icold = lgb_model.predict(X_test_ucold_icold_transformed)

In [ ]:
# Evaluate

# Calculate MSE for each test set
mse_uwarm_iwarm = mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mse_uwarm_icold = mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mse_ucold_iwarm = mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mse_ucold_icold = mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)

# Calculate RMSE for each test set
rmse_uwarm_iwarm = root_mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
rmse_uwarm_icold = root_mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
rmse_ucold_iwarm = root_mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
rmse_ucold_icold = root_mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)

# Calculate MAE for each test set
mae_uwarm_iwarm = mean_absolute_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mae_uwarm_icold = mean_absolute_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mae_ucold_iwarm = mean_absolute_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mae_ucold_icold = mean_absolute_error(y_test_ucold_icold, y_pred_ucold_icold)

# Print the results
print(f'MSE for warm user warm item: {mse_uwarm_iwarm:.4f}')
print(f'RMSE for warm user warm item: {rmse_uwarm_iwarm:.4f}')
print(f'MAE for warm user warm item: {mae_uwarm_iwarm:.4f}')
print('-' * 50)
print(f'MSE for warm user cold item: {mse_uwarm_icold:.4f}')
print(f'RMSE for warm user cold item: {rmse_uwarm_icold:.4f}')
print(f'MAE for warm user cold item: {mae_uwarm_icold:.4f}')
print('-' * 50)
print(f'MSE for cold user warm item: {mse_ucold_iwarm:.4f}')
print(f'RMSE for cold user warm item: {rmse_ucold_iwarm:.4f}')
print(f'MAE for cold user warm item: {mae_ucold_iwarm:.4f}')
print('-' * 50)
print(f'MSE for cold user cold item: {mse_ucold_icold:.4f}')
print(f'RMSE for cold user cold item: {rmse_ucold_icold:.4f}')
print(f'MAE for cold user cold item: {mae_ucold_icold:.4f}')

In [ ]:
# XGBoost with DMatrix
dtrain = xgb.DMatrix(X_train_transformed, label=y_train, feature_names=list(feature_names))
xgb_model = xgb.train(
    params={
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'seed': 42
    },
    dtrain=dtrain,
    num_boost_round=100
)

d_test_uwarm_iwarm = xgb.DMatrix(X_test_uwarm_iwarm_transformed, feature_names=list(feature_names))
d_test_uwarm_icold = xgb.DMatrix(X_test_uwarm_icold_transformed, feature_names=list(feature_names))
d_test_ucold_iwarm = xgb.DMatrix(X_test_ucold_iwarm_transformed, feature_names=list(feature_names))
d_test_ucold_icold = xgb.DMatrix(X_test_ucold_icold_transformed, feature_names=list(feature_names))

# Predict using the trained model
y_pred_uwarm_iwarm = xgb_model.predict(d_test_uwarm_iwarm)
y_pred_uwarm_icold = xgb_model.predict(d_test_uwarm_icold)
y_pred_ucold_iwarm = xgb_model.predict(d_test_ucold_iwarm)
y_pred_ucold_icold = xgb_model.predict(d_test_ucold_icold)


In [ ]:
# Evaluate
# Calculate MSE for each test set
mse_uwarm_iwarm = mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mse_uwarm_icold = mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mse_ucold_iwarm = mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mse_ucold_icold = mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)
# Calculate RMSE for each test set
rmse_uwarm_iwarm = root_mean_squared_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
rmse_uwarm_icold = root_mean_squared_error(y_test_uwarm_icold, y_pred_uwarm_icold)
rmse_ucold_iwarm = root_mean_squared_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
rmse_ucold_icold = root_mean_squared_error(y_test_ucold_icold, y_pred_ucold_icold)
# Calculate MAE for each test set
mae_uwarm_iwarm = mean_absolute_error(y_test_uwarm_iwarm, y_pred_uwarm_iwarm)
mae_uwarm_icold = mean_absolute_error(y_test_uwarm_icold, y_pred_uwarm_icold)
mae_ucold_iwarm = mean_absolute_error(y_test_ucold_iwarm, y_pred_ucold_iwarm)
mae_ucold_icold = mean_absolute_error(y_test_ucold_icold, y_pred_ucold_icold)
# Print the results
print(f'MSE for warm user warm item: {mse_uwarm_iwarm:.4f}')
print(f'RMSE for warm user warm item: {rmse_uwarm_iwarm:.4f}')
print(f'MAE for warm user warm item: {mae_uwarm_iwarm:.4f}')
print('-' * 50)
print(f'MSE for warm user cold item: {mse_uwarm_icold:.4f}')
print(f'RMSE for warm user cold item: {rmse_uwarm_icold:.4f}')
print(f'MAE for warm user cold item: {mae_uwarm_icold:.4f}')
print('-' * 50)
print(f'MSE for cold user warm item: {mse_ucold_iwarm:.4f}')
print(f'RMSE for cold user warm item: {rmse_ucold_iwarm:.4f}')
print(f'MAE for cold user warm item: {mae_ucold_iwarm:.4f}')
print('-' * 50)
print(f'MSE for cold user cold item: {mse_ucold_icold:.4f}')
print(f'RMSE for cold user cold item: {rmse_ucold_icold:.4f}')
print(f'MAE for cold user cold item: {mae_ucold_icold:.4f}')
